In [ ]:
pip install --upgrade openai

In [ ]:
import json
from openai import OpenAI
import os
import pandas as pd

In [ ]:
client = OpenAI(api_key="")

In [ ]:
results = ".../Results"
files = {
        "spec1_random_500_20251110_180802.xlsx": "spec1_random_500",
    "spec2_recent_500_20251110_181330.xlsx": "spec2_recent_500",
    "spec3_recent_500_plus_10q_20251110_182010.xlsx": "spec3_recent_500_plus_10q",
    "spec4_persona_plus_10q_20251110_182428.xlsx": "spec4_persona_plus_10q",
}

gen_col = "generated_answer"

audio_root = os.path.join(results, "audio_onyx")

max_rows = None

os.makedirs(audio_root, exist_ok = True)

for filename, specname in files.items():
    path = os.path.join(results, filename)
    if not os.path.exists(path):
        print(f"File not found, skipping: {path}")
        continue
    print(f"\n=== Processing {filename} ({specname}) ===")
    df = pd.read_excel(path)
    if gen_col not in df.columns:
        raise ValueError(
            f"Column '{gen_col}' not found in {filename}"
            f"Available columns: {list(df.columns)}"
        )
    
    out_dir = os.path.join(audio_root, specname)
    os.makedirs(out_dir, exist_ok=True)
    print(f"Audio going to be written to: {out_dir}")
    
    has_quarter = "quarter" in df.columns
    has_id = "test_id" in df.columns
    num_rows = len(df)
    print(f"Rows: {num_rows}")
    
    for idx, row in df.iterrows():
        if max_rows is not None and idx >= max_rows:
            break
        text = str(row[gen_col]).strip()
        if not text:
            continue
        if has_quarter and has_id:
            quarter = str(row["quarter"])
            test_id = int(row["test_id"])
            base_name = f"{quarter}_test{test_id:02d}"
        else:
            base_name = f"row_{idx:03d}"
        out_path = os.path.join(out_dir, base_name + ".mp3")
        if os.path.exists(out_path):
            continue
        print(f"[TTS ] {specname}: {base_name}")
              
        audio_resp = client.audio.speech.create(
            model = "gpt-4o-mini-tts",
            voice = "onyx",
            input =text,
        )
        with open(out_path, "wb") as f:
              f.write(audio_resp.read())
    